# 开发情况记录
- 2018-10-30 （1）基本文件读入内存 OK （2）对标人行数据筛查功能 雏形验证 
- 2018-10-31 （1）考虑基本的程序功能组合并采用合理方式表示 考虑传入变长参数
- 2018-11-01 （2）为程序的灵活性，考虑query问题

In [14]:
#------------引入库包----------------
#import os
import gc #释放内存用
import pandas as pd
from pandasql import sqldf


def pysqldf(query):
   
    #使用该函数后可以不再添加locals()参数
    
    return sqldf(query, globals())

#------------赋值常量----------------
CONST_ROWS = 1000 # 读入csv文件时，每次读入CONST_ROWS行处理，避免内存溢出


#-------------先释放前期内存-----------
if 'reader' in locals().keys():
    del reader
    gc.collect()
    print("删除reader")
else:
    print("建立reader")
    
if 'df' in locals().keys():
    del df
    gc.collect()
    print("删除df")
else:
    print("建立df")

if 'chunks' in locals().keys():
    del chunks
    gc.collect()
    print("删除chunks")
else:
    print("建立表2")
    

#for key in list(locals().keys()):
#	del locals()[key]
#print('1')
#-------------建立文件对象并读文件-----------
with open('G:/aml/绵阳/提数原始数据/new4.csv','r',encoding='gb18030') as f_read:
#f_read = open('G:/aml/绵阳/提数原始数据/new4.csv','r',encoding='gb18030')
    reader = pd.read_csv(f_read,#此处无法使用f_read.read(size)函数，原因是在pd.read_csv函数中的参数限制了该地方的使用
                         sep = '|',#分割符是:|
                         
                         #chunksize = CONST_ROWS,
                         #使用本参数后则pd.read_csv返回迭代类型TextFileReader给reader，每CONST_ROWS条迭代一次，数据占用较大内存时使用
                         #那么在接下来的代码中需要将reader迭代才能完整表达原csv文件内容
                         iterator=True, #使用迭代，不能与此chunksize同时使用？
                         
                         #———————————以下赋列名（按照人行标准文件接口定义列名）——————————
                         header = None,#与下面names配套使用
                         names = ['Head_no',
                                'Bank_code1',
                                'Cst_no',
                                'Open_time',
                                'Close_time',
                                'Acc_name',
                                'Cst_sex',
                                'Nation',
                                'Id_type',
                                'Id_no',
                                'Id_deadline',
                                'Occupation',
                                'Income',
                                'Contact1',
                                'Contact2',
                                'Contact3',
                                'Address1',
                                'Address2',
                                'Address3',
                                'Company',
                                'Sys_name']
                        )

    #————————————————以下需要对reader迭代处理——————————————————
    chunks = [] #初始化块
#    for r in reader:
#        chunk = r.get_chunk(CONST_ROWS)#get_chunk()必须输入参数表示读多少行？
#        chunks.append(chunk)  
    
#print("Iteration is stopped.")
#df = pd.concat(chunks, ignore_index=True)
#a=(11,12)性别元组？

    loop = True
    while loop:
        try:
            chunk = reader.get_chunk(CONST_ROWS)
            #chunk2 = chunk[(chunk['Id_deadline']< 20180930)]#筛查出身份证件有效期在20180930之前的客户记录
            #chunk2 = chunk.query('Id_deadline < 20180930') #验证query函数的使用，1、此处为身份证件到期
            
            #chunk2 = chunk.query('Id_no.length() == 15') # 2、一代身份证,未通过，使用query似乎不行
            #chunk2 = chunk[chunk['Id_no'].str.len() == 15]
        
            #考虑引入pandasql库后采用sql语言
            
            # 1、身份证件有效期在20180930之前的客户记录
            #chunk2 = pysqldf('select * from chunk where Id_deadline < 20180930;') 
            
            # 2、一代身份证
            #chunk2 = pysqldf('select * from chunk where length(Id_no)=15;') 
            
            # 3、联系方式为空
            #chunk2 = pysqldf('select * from chunk where Contact1 is null and Contact2 is null and Contact3 is null;') 
            
            # 4、年龄46岁以下为永久身份证
            #chunk2 = pysqldf('select * from chunk where Id_type=11 and ((Id_deadline=99991231) or (Id_deadline=20991231)) \
            #and ((length(Id_no)=15 and (2018-1900-substr(Id_no,7,2))<46) or (length(Id_no)=18 and (2018-substr(Id_no,7,4))<46));')
            
            # 5、身份证号码为错误
            #chunk2 = pysqldf('select * from chunk where length(Id_no)<>15 and length(Id_no)<>18 and Id_type=11;') 
            
            # 6、职业为空
            #chunk2 = pysqldf('select * from chunk where Occupation is null;') 
            
            # 7、性别为空(或者错误？)
            #chunk2 = pysqldf('select * from chunk where Cst_sex is null;') 
            
            # 8、身份证件类型为空(或者错误?)
            #chunk2 = pysqldf('select * from chunk where Id_type is null;')
            #chunk2 = pysqldf('select * from chunk where Id_type not in(11,12,13,14,19);')
            
            
            chunks.append(chunk2)
            #del chunk
            del chunk2
        except StopIteration:
            loop = False
            print("OK")
    df = pd.concat(chunks, ignore_index=True)#ignore_index 忽略需要连接的frame本身的index。当原本的index没有特别意义的时候可以使用
df.shape




删除reader
建立df
删除chunks
OK


(2834, 21)

# 筛查逻辑
## 一、主体结构抽象
### 1.输入
- 
### 2.输出
### 3.实现
## 二、具体筛查模块

In [6]:
! pip install pandasql

  Running setup.py bdist_wheel for pandasql: started
  Running setup.py bdist_wheel for pandasql: finished with status 'done'
  Stored in directory: C:\Users\qlx\AppData\Local\pip\Cache\wheels\53\6c\18\b87a2e5fa8a82e9c026311de56210b8d1c01846e18a9607fc9
Successfully built pandasql


In [4]:
pd.set_option('display.max_columns', None)
df.head()

,Head_no,Bank_code1,Cst_no,Open_time,Close_time,Acc_name,Cst_sex,Nation,Id_type,Id_no,Id_deadline,Occupation,Income,Contact1,Contact2,Contact3,Address1,Address2,Address3,Company,Sys_name
0,10011101026226,23084151,230800002409275,20020511,99991231,陶彦辉,11,CHN,11,5101026412260803,99991231,无职业,0.0,13699600697,None,None,绵阳游仙绵阳市绵山路64号,None,None,None,企业级客户信息
1,10011101026226,23084151,230800001549044,20020511,99991231,陈平,11,CHN,11,5107026403220933,99991231,无职业,0.0,2489004,None,None,绵阳游仙绵阳市绵山路64号,None,None,None,企业级客户信息
2,10011101026226,23084151,230800002217146,20020511,99991231,马军,11,CHN,11,5107027211190933,99991231,事业单位员工,0.0,13658116722,None,None,绵阳市游仙区绵山路64号9区12栋1单元502,None,None,None,企业级客户信息
3,10011101026226,23084151,230800002318322,20020511,99991231,邵文普,11,CHN,11,5107026611130933,20270309,自由职业者,0.0,15984651331,None,None,绵阳游仙科学城六区,None,None,None,企业级客户信息
4,10011101026226,23084151,230800002787207,20020511,99991231,于泊洋,12,CHN,11,5107027411130934,99991231,事业单位员工,0.0,64111364,None,None,绵阳市游仙区科学城1所,None,None,None,企业级客户信息


In [1]:
#2018-11-04测试git
print('测试提交git')

测试提交git


In [9]:
pd.options.display.max_rows

60

In [3]:
pd.set_option('display.max_columns', None)